<a href="https://colab.research.google.com/github/ash-victor/Food-Access-NC/blob/main/SQL_lite_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set-Up

In [ ]:
import csv
import sqlite3

In [ ]:
conn = sqlite3.connect('Food_Access_Atlas.db')

In [ ]:
cursor = conn.cursor()

In [ ]:
# SQLite: Loading Food Access Database
filepath = "drive/My Drive/projects/Food_Access/Data/Food_AccessNC.csv"

In [ ]:
with open(filepath, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # Read the header row

    # Create a table with columns based on the CSV header
    create_table_query = f'''
        CREATE TABLE IF NOT EXISTS Food_Access (
            {', '.join([f'{column} TEXT' for column in header])}
        )
    '''
    cursor.execute(create_table_query)

    # Insert data into the table
    insert_query = f'''
        INSERT INTO Food_Access ({', '.join(header)})
        VALUES ({', '.join(['?' for _ in header])})
    '''

    for row in csv_reader:
        cursor.execute(insert_query, row)

# Commit changes and close the connection
conn.commit()


In [ ]:
# Execute a SELECT query to retrieve data from the table
cursor.execute('SELECT * FROM Food_Access')

# Fetch all the rows from the query result
rows = cursor.fetchall()

# Print the column headers
header = [description[0] for description in cursor.description]
print("\t".join(header))

# Print the data
for row in rows:
    print("\t".join(map(str, row)))



In [ ]:
# Get column names from the table
columns = [description[1] for description in cursor.execute('PRAGMA table_info(Food_Access)').fetchall()]

# Update each column to replace "NULL" with NULL values
for column in columns:
    update_query = f'UPDATE Food_Access SET {column} = NULL WHERE {column} = "NULL"'
    cursor.execute(update_query)

# Commit changes and close the connection
conn.commit()

In [ ]:
# Fetch and print data from the updated table
cursor.execute(f'SELECT * FROM Food_Access')
rows = cursor.fetchall()

# Print the column headers
header = columns
print("\t".join(header))

# Print the data
for row in rows:
    print("\t".join(map(str, row)))

### Analyzing NC Counties and SNAP Benefits

In [ ]:
# Fields about SNAP benefits:
# lasnaphalf = Low access, housing units receiving SNAP benefits at 1/2 mile, number
# lasnaphalfshare = Low access, housing units receiving SNAP benefits at 1/2 mile, share
# lasnap1 = Low access, housing units receiving SNAP benefits at 1 mile, number
# lasnap1share = Low access, housing units receiving SNAP benefits at 1 mile, share
# lasnap10 = Low access, housing units receiving SNAP benefits at 10 miles, number
# lasnap10share = Low access,housing units receiving SNAP benefits at 10 miles, share
# lasnap20 = Low access, housing units receiving SNAP benefits at 20 miles, number
# lasnap20share = Low access, housing units receiving SNAP benefits at 20 miles, share

In [ ]:
# Query 1: What Census Tract has the lowest number of housing units recieving SNAP benefits?
# Query 2: What Census Tract has the highest number of housing units recieving SNAP benefits?

# TractSNAP : Tract housing units receiving SNAP benefits, number

# Find the CensusTract and County with the lowest and highest TractSNAP values
min_max_query = 'SELECT CensusTract, County FROM Food_Access WHERE TractSNAP = (SELECT MIN(TractSNAP) FROM Food_Access) OR TractSNAP = (SELECT MAX(TractSNAP) FROM Food_Access)'
cursor.execute(min_max_query)
min_max_tracts = cursor.fetchall()

# Print the result
print("CensusTract with Lowest TractSNAP:", min_max_tracts[0][0], "in County:", min_max_tracts[0][1])
print("CensusTract with Highest TractSNAP:", min_max_tracts[1][0], "in County:", min_max_tracts[1][1])


CensusTract with Lowest TractSNAP: 37001021701 in County: Alamance County
CensusTract with Highest TractSNAP: 37031970604 in County: Carteret County


In [ ]:
# Query 3: Need to make a visualization of the top 10 counties with the highest number of housing units utilizing SNAP benefits

# Commit changes

# Find the top 10 counties with the highest TractSNAP values
top_counties_query = 'SELECT County, MAX(TractSNAP) AS MaxTractSNAP FROM Food_Access GROUP BY County ORDER BY MaxTractSNAP DESC LIMIT 10'
cursor.execute(top_counties_query)
top_counties = cursor.fetchall()

# Print the result
print("Top 10 Counties with Highest TractSNAP:")
print("County\tMax TractSNAP")
for row in top_counties:
    county, max_tract_snap = row
    print(f'{county}\t{max_tract_snap}')

Top 10 Counties with Highest TractSNAP:
County	Max TractSNAP
Wake County	994
Mecklenburg County	99
Guilford County	99
Davidson County	99
Onslow County	98
Iredell County	98
Forsyth County	98
Durham County	98
Surry County	97
Rowan County	97


In [ ]:
# Query 4: Need to make a visualization of the top 10 counties with the lowest number of housing units utilizing SNAP benefits

# Commit changes

# Find the top 10 counties with the lowest TractSNAP values
top_counties_query = 'SELECT County, MIN(TractSNAP) AS MinTractSNAP FROM Food_Access GROUP BY County ORDER BY MinTractSNAP DESC LIMIT 10'
cursor.execute(top_counties_query)
top_counties = cursor.fetchall()

# Print the result
print("Top 10 Counties with Lowest TractSNAP:")
print("County\tMin TractSNAP")
for row in top_counties:
    county, max_tract_snap = row
    print(f'{county}\t{max_tract_snap}')

Top 10 Counties with Lowest TractSNAP:
County	Min TractSNAP
Hyde County	386
Bertie County	369
Tyrrell County	314
Bladen County	299
Greene County	291
Clay County	289
Perquimans County	278
Warren County	261
Sampson County	243
Ashe County	242


In [ ]:
# Query 5: Need to make a visualization of the top 10 CensusTract with the highest number of housing units utilizing SNAP benefits

# Commit changes

# Find the top 10 Census Tracts with the highest TractSNAP values
top_counties_query = 'SELECT CensusTract, MAX(TractSNAP) AS MaxTractSNAP FROM Food_Access GROUP BY CensusTract ORDER BY MaxTractSNAP DESC LIMIT 10'
cursor.execute(top_counties_query)
top_counties = cursor.fetchall()

# Print the result
print("Top 10 Census Tracts with Highest TractSNAP:")
print("CensusTract\tMax TractSNAP")
for row in top_counties:
    census, max_tract_snap = row
    print(f'{census}\t{max_tract_snap}')

Top 10 Census Tracts with Highest TractSNAP:
CensusTract	Max TractSNAP
37183052806	994
37119005523	99
37119002100	99
37081012508	99
37057060102	99
37183053420	98
37133000900	98
37097060703	98
37067002804	98
37063001806	98


In [ ]:
# Query 6: How many Counties in North Carolina have more than 800 housing units utilizing SNAP benefits?
# Execute the query
count_query = 'SELECT COUNT(DISTINCT County) FROM Food_Access WHERE TractSNAP > 800 AND State = "North Carolina"'
cursor.execute(count_query)
result = cursor.fetchone()

# Print the result
count_of_counties = result[0]
print(f'The number of counties in North Carolina with more than 800 housing units utilizing SNAP benefits is: {count_of_counties}')



The number of counties in North Carolina with more than 800 housing units utilizing SNAP benefits is: 44


### Analyzing NC Counties and Vehicle Access

In [ ]:
# Fields about Vehicle Access:
# HUNVFlag = Vehicle access, tract with low vehicle access (Flag for tract where >= 100 of households do not have a vehicle, and beyond 1/2 mile from supermarket)
# lahunv10 = Vehicle access, housing units without and low access at 10 miles, number
# lahunv10share = Vehicle access, housing units without and low access at 10 miles, share
# lahunv20 = Vehicle access, housing units without and low access at 20 miles, number
# lahunv20share = Vehicle access, housing units without and low access at 20 miles, share
# TractHUNV = Tract housing units without a vehicle, number

### Income

In [ ]:
# Fields about Income:
#

### Children

In [ ]:
# Fields about Children

### Seniors

In [ ]:
# Fields about Seniors

### White

In [ ]:
# Fields about white people

### Black

In [ ]:
# Fields about black people

### Asian

In [ ]:
# Fields about asian people

### Native Hawaiian and Other Pacific Islander population

### Latino

### American Indian and Alaska Native

In [ ]:
# Fields about latino people

In [ ]:
# Close the connection
conn.close()